# Associate IAM Policies with EKS Worker Nodes


In [1]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=
export INSTANCE_PROFILE_ARN=


# Attach Roles
* Allow Access from/to the Elastic Container Registry (ECR)
* This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR. 
* We will load these custom Docker images in a later section.

In [2]:
%%bash

source ~/.bash_profile

aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take 5-10 minutes.  Please be patient._

# _If you see `retryable error`s below, this is OK!_

In [3]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve

echo "Completed"

[ℹ]  eksctl version 0.28.1
[ℹ]  using region us-west-2
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 36.32453ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 83.691154ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 233.257076ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiti

# _If you see `retryable error`s ^^ above ^^, this is OK!_

In [4]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.us-west-2.amazonaws.com/id/A5C0EB6CD6100B8008E1B68132B20A27
Completed


# Update `~/.kube/config` with our new EKS cluster

In [5]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-west-2:354474215062:cluster/cluster to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [6]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   4m6s
kube-node-lease   Active   4m8s
kube-public       Active   4m8s
kube-system       Active   4m8s


In [7]:
%%bash

kubectl get nodes


No resources found.


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();